In [1]:
from datetime import datetime

from pymongo import MongoClient
import pandas as pd

import constants
from tqdm import tqdm


In [2]:
mongo = MongoClient(constants.mongo_conn_string)
mongo_lt_col = mongo[constants.mongo_db][constants.mongo_lv_annotated_col]

In [3]:
query = {}

cursor = mongo_lt_col.find(query, {"_id":0, "text":0})

data = [i for i in cursor]
print(len(data))

df = pd.DataFrame(data)
print(df.shape)
df.head()

28864
(28864, 10)


,para_id,source,cabinet_ents,date,year,month,day,cabinet,cabinet_no,president
0,61d01db0c36b93aa8c582d24,https://www.delfi.lv/news/national/politics/va...,[Aizsardzības],1999-11-25 02:33:00,1999,11,25,Andris Šķēle,27,Vaira Viķe Freiberga
1,61cf823fc36b93aa8c57704e,https://www.delfi.lv/news/national/politics/be...,[Iekšlietu],1999-11-25 13:09:00,1999,11,25,Andris Šķēle,27,Vaira Viķe Freiberga
2,61cffbcfc36b93aa8c580460,https://www.delfi.lv/news/national/politics/st...,[Premjers],1999-11-25 17:16:00,1999,11,25,Andris Šķēle,27,Vaira Viķe Freiberga
3,61cfad4fc36b93aa8c57a4eb,https://www.delfi.lv/news/national/politics/ce...,[Premjers],1999-11-25 18:51:00,1999,11,25,Andris Šķēle,27,Vaira Viķe Freiberga
4,61cfd538c36b93aa8c57d5aa,https://www.delfi.lv/news/national/politics/sk...,[Premjers],1999-11-25 18:55:00,1999,11,25,Andris Šķēle,27,Vaira Viķe Freiberga


In [4]:
df.to_parquet("DataLV.parquet")

# Basic Data

In [5]:
df.year.value_counts()

2011    2663
2009    2587
2010    1894
2012    1811
2008    1765
2021    1669
2007    1662
2013    1555
2015    1464
2016    1341
2014    1192
2006    1118
2020    1084
2019    1054
2005    1000
2018     956
2017     865
2003     783
2004     775
2002     541
2000     388
2001     337
2022     309
1999      51
Name: year, dtype: int64

In [7]:
df.cabinet_no.value_counts()

40    4032
36    3877
34    3636
39    3056
35    2619
33    2320
31    2035
38    1854
32    1769
29    1060
28    1002
37     866
30     535
27     203
Name: cabinet_no, dtype: int64

In [8]:
df.president.value_counts()

Valdis Zatlers          8740
Andris Bērziņš          6522
Vaira Viķe Freiberga    5660
Raimonds Vējonis        4394
Eglis Levits            3548
Name: president, dtype: int64

# Adding dummies for media & presidential attention for each policy sphere

In [11]:
ministries = set()
for i in list(df.cabinet_ents):
    for a in i:
        ministries.add(a)
    
for item in tqdm(list(ministries)):
    df[item]= df.apply(lambda x : int(item in x["cabinet_ents"]), axis = 1)


def prez_attention( list_min , minister):
    if "Prezidents" in list_min and minister in list_min:
        return 1
    else:
        return 0
    
for item in tqdm(list(ministries)):
    if item != "Prezidenta":
        df["president_"+item]= df.apply(lambda x : prez_attention(x["cabinet_ents"], item), axis = 1)
    
df.head() 

100%|██████████| 19/19 [00:07<00:00,  2.46it/s]


,para_id,source,cabinet_ents,date,year,month,day,cabinet,cabinet_no,president,...,president_Ārlietu,president_Izglītības un zinātnes,president_Aizsardzības,president_Finanšu,president_Vides aizsardzības un reģionālās attīstības,president_Reģionālās attīstības un pašvaldību lietu,president_Kultūras,president_Prezidents,president_Ekonomikas,president_Labklājības
0,61d01db0c36b93aa8c582d24,https://www.delfi.lv/news/national/politics/va...,[Aizsardzības],1999-11-25 02:33:00,1999,11,25,Andris Šķēle,27,Vaira Viķe Freiberga,...,0,0,0,0,0,0,0,0,0,0
1,61cf823fc36b93aa8c57704e,https://www.delfi.lv/news/national/politics/be...,[Iekšlietu],1999-11-25 13:09:00,1999,11,25,Andris Šķēle,27,Vaira Viķe Freiberga,...,0,0,0,0,0,0,0,0,0,0
2,61cffbcfc36b93aa8c580460,https://www.delfi.lv/news/national/politics/st...,[Premjers],1999-11-25 17:16:00,1999,11,25,Andris Šķēle,27,Vaira Viķe Freiberga,...,0,0,0,0,0,0,0,0,0,0
3,61cfad4fc36b93aa8c57a4eb,https://www.delfi.lv/news/national/politics/ce...,[Premjers],1999-11-25 18:51:00,1999,11,25,Andris Šķēle,27,Vaira Viķe Freiberga,...,0,0,0,0,0,0,0,0,0,0
4,61cfd538c36b93aa8c57d5aa,https://www.delfi.lv/news/national/politics/sk...,[Premjers],1999-11-25 18:55:00,1999,11,25,Andris Šķēle,27,Vaira Viķe Freiberga,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df.columns

Index(['para_id', 'source', 'cabinet_ents', 'date', 'year', 'month', 'day',
       'cabinet', 'cabinet_no', 'president',
       'Bērnu, ģimenes un sabiedrības integrācijas lietu', 'Veselības',
       'Premjers', 'Zemkopības', 'Bērnu un ģimenes lietu', 'Satiksmes',
       'Iekšlietu', 'Tieslietu', 'Vides', 'Ārlietu', 'Izglītības un zinātnes',
       'Aizsardzības', 'Finanšu',
       'Vides aizsardzības un reģionālās attīstības',
       'Reģionālās attīstības un pašvaldību lietu', 'Kultūras', 'Prezidents',
       'Ekonomikas', 'Labklājības',
       'president_Bērnu, ģimenes un sabiedrības integrācijas lietu',
       'president_Veselības', 'president_Premjers', 'president_Zemkopības',
       'president_Bērnu un ģimenes lietu', 'president_Satiksmes',
       'president_Iekšlietu', 'president_Tieslietu', 'president_Vides',
       'president_Ārlietu', 'president_Izglītības un zinātnes',
       'president_Aizsardzības', 'president_Finanšu',
       'president_Vides aizsardzības un reģionālās at

In [13]:
df.to_parquet("DataEnhanced_LV.parquet")

In [14]:
df.to_csv("DataEnhanced_LV.csv")